In [ ]:
!pip install ultralytics

     -------------------------------------- 752.1/752.1 kB 5.9 MB/s eta 0:00:00
     --------------------------------------- 38.6/38.6 MB 14.6 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\79252\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from ultralytics import YOLO
import cv2
import math
import numpy as np

In [2]:

model = YOLO('yolov8n-seg.pt')  # load a pretrained model (recommended for training)


In [3]:
blue_color = (255, 0, 0)

In [9]:
cap = cv2.VideoCapture(0)
cap.set(3, 640) # Set width
cap.set(4, 480) # Set height
while True:
    success, img = cap.read()
    results = model(img, stream=True, classes = 0)
    # masks = (results.masks)
    for r in results:
        
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)


            cls = int(box.cls[0])
            # cv2.putText(img, class_names[cls], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    # for mask in masks:
            # background_mask = cv2.bitwise_not(mask.xy)
            # print()
        # colored_mask = np.expand_dims(mask, 0).repeat(3, axis=0)
        
        # background_mask = cv2.bitwise_not(mask.data[0].numpy())
        # img[np.where(background_mask)] = blue_color
        # print(background_mask)
    

    cv2.imshow('Webcam', img)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 480x640 2 persons, 210.6ms
Speed: 9.0ms preprocess, 210.6ms inference, 809.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 142.6ms
Speed: 3.0ms preprocess, 142.6ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 128.0ms
Speed: 2.0ms preprocess, 128.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 138.0ms
Speed: 2.0ms preprocess, 138.0ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 168.0ms
Speed: 2.0ms preprocess, 168.0ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 182.0ms
Speed: 6.0ms preprocess, 182.0ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 166.8ms
Speed: 1.0ms preprocess, 166.8ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 191.0ms
Speed: 3.0ms preprocess, 191.0ms inference, 8.0ms postprocess pe

In [ ]:
cap.release()
cv2.destroyAllWindows()